In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall dill
!pip install dill==0.3.4

Found existing installation: dill 0.3.6
Uninstalling dill-0.3.6:
  Would remove:
    /usr/local/bin/get_objgraph
    /usr/local/bin/undill
    /usr/local/lib/python3.8/dist-packages/dill-0.3.6.dist-info/*
    /usr/local/lib/python3.8/dist-packages/dill/*
Proceed (y/n)? y
  Successfully uninstalled dill-0.3.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.4 which is incompatible.
evaluate 0.3.0 requires datasets>=2.0.0, but you have datasets 1.0.2 which is incompatible.


## Train

In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score

In [ ]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/ 
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 32.2 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645950 sha256=0344982abf1777f4a55896dd8e6896a922a5fb1fdebfcf74e02156cdeec9d4c2
  Stored in directory: /root/.cache/pip/wheels/3c/e9/86/706911c12e307aeb9a2702399f0dad38d36f1d6f9dde8af35e
Successfully built vncorenlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 10.5 MB/s 
  Attempting uninstall: datasets
    Found existing installation: datasets 2.7.1
    Uninstalling datasets-2.7.1:
      Successfully uninstalled datasets-2.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
evaluate 0.3.0 requires datasets>=2.0.0, but you have dat

In [ ]:
!gsutil cp -r gs://vietai_public/viT5/data/vietnews .
!gsutil cp -r gs://vietai_public/viT5/data/wikilingua .

Copying gs://vietai_public/viT5/data/vietnews/test.tsv...
Copying gs://vietai_public/viT5/data/vietnews/test_dedup.tsv...
Copying gs://vietai_public/viT5/data/vietnews/train_dedup.tsv...
- [3 files][422.4 MiB/422.4 MiB]   45.3 MiB/s                                   
Operation completed over 3 objects/422.4 MiB.                                    
Copying gs://vietai_public/viT5/data/wikilingua/test.tsv...
Copying gs://vietai_public/viT5/data/wikilingua/train.tsv...
Copying gs://vietai_public/viT5/data/wikilingua/val.tsv...
/ [3 files][ 63.5 MiB/ 63.5 MiB]                                                
Operation completed over 3 objects/63.5 MiB.                                     


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments, AutoModel
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")

#tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
#model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")


model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36100, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36100, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    labels = tokenizer(
        examples["labels"], max_length=256, truncation=True, padding=True
    )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
#data = pd.read_excel("/content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/data_Tuoi_Tre.xlsx")
#data = pd.read_excel("/content/drive/MyDrive/data_Tuoi_Tre.xlsx")
#data = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/data_Tuoi_Tre.xlsx")
data = pd.read_excel("/content/drive/MyDrive/NLP/Data/data_Tuoi_Tre.xlsx")

In [ ]:
#data = data.iloc[:100, :]

In [ ]:
input_lines = data["Content"].values
label_lines = data["Summary"].values

for i in range(len(input_lines)):
  input_lines[i] = str(input_lines[i]) + '</s>'

dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)



In [ ]:

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

"""/content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/ViT5/Base/"""
"""/content/drive/MyDrive/NLP/Model"""
training_args = Seq2SeqTrainingArguments("temp/",
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=3,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="steps",
                                      save_steps= 2000,  
                                      save_total_limit=10,
                                      overwrite_output_dir=True,
                                      fp16=True,
                                      )


# AdaFactor for ViT5-large models as it based on T5v1.1.
# See https://medium.com/the-artificial-impostor/paper-adafactor-adaptive-learning-rates-with-sublinear-memory-cost-a543abffa37

# from transformers.optimization import Adafactor, AdafactorSchedule
# optimizer = Adafactor(
#     model.parameters(),
#     lr=1e-3,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False
# )
# lr_scheduler = AdafactorSchedule(optimizer)


In [ ]:
import torch
torch.cuda.empty_cache()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler)
)

trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 23326
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 17496
  Number of trainable parameters = 791284736
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: ignored

In [ ]:
import torch
#torch.cuda.empty_cache()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    optimizers=(optimizer, lr_scheduler)
)
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/ViT5/Base/checkpoint-150")

Using cuda_amp half precision backend
Loading model from /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/ViT5/Base/checkpoint-150.
***** Running training *****
  Num examples = 100
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 250
  Number of trainable parameters = 225950976
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 6
  Continuing training from global step 150
  Will skip the first 6 epochs then the first 0 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


0it [00:00, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


Saving model checkpoint to /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/ViT5/Base/checkpoint-175
Configuration saved in /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/ViT5/Base/checkpoint-175/config.json
Model weights saved in /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/ViT5/Base/checkpoint-175/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/ViT5/Base/checkpoint-200
Configuration saved in /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/ViT5/Base/checkpoint-200/config.json
Model weig

KeyboardInterrupt: ignored

## Inference

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [ ]:
input_lines = []
label_lines = []
task = 'wikilingua'
with open(f'{task}/test.tsv') as file:
  for line in file:
    line = line.strip().split('\t')
    input = line[0]
    input_lines.append(input +'</s>')
    label_lines.append(line[1])



input_lines  = input_lines
label_lines = label_lines
dict_obj = {'inputs': input_lines, 'labels': label_lines}

dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=10)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/NLP/Model/checkpoint-16000")
model.to('cuda')

loading configuration file /content/drive/MyDrive/NLP/Model/checkpoint-16000/config.json
Model config T5Config {
  "_name_or_path": "/content/drive/MyDrive/NLP/Model/checkpoint-16000",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 36096
}

loading weights file /content/drive/MyDrive/NLP/Model/checkpoint-16000/pytorch_model.bin
All model checkpoint weig

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
import torch 
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()


  0%|          | 0/123 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'rouge1': AggregateScore(low=Score(precision=0.6139732329884544, recall=0.4707550452679877, fmeasure=0.490701776710809), mid=Score(precision=0.6200810794467819, recall=0.4761418568071456, fmeasure=0.49435190055576556), high=Score(precision=0.626197765221189, recall=0.4813727026676934, fmeasure=0.49803555606896144)),
 'rouge2': AggregateScore(low=Score(precision=0.22168745990026903, recall=0.16566939270525752, fmeasure=0.172571910180997), mid=Score(precision=0.22558155618546338, recall=0.1690024807872715, fmeasure=0.17533552716059633), high=Score(precision=0.22957451099079634, recall=0.1723422641079218, fmeasure=0.17784907803575173)),
 'rougeL': AggregateScore(low=Score(precision=0.3782505538095582, recall=0.28967958103000296, fmeasure=0.29980483379254164), mid=Score(precision=0.3821010660720794, recall=0.2932446078351624, fmeasure=0.30174712657913466), high=Score(precision=0.38627183262747766, recall=0.29677676344863424, fmeasure=0.30400974159912)),
 'rougeLsum': AggregateScore(low=Sc

In [ ]:
inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")
attention_mask = inputs.attention_mask.to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)

# all special tokens including will be removed
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str

NameError: ignored

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]


[{'rouge1': 0.49435190055576556},
 {'rouge2': 0.17533552716059633},
 {'rougeL': 0.30174712657913466},
 {'rougeLsum': 0.3017289079226135}]

In [ ]:
i = 10
print('Prediction: ',predictions[i])
print('Truth: ',label_lines[i])
print('Content: ',input_lines[i])

Prediction:  Một số người thường có suy nghĩ và hành động hoảng loạn khi đối mặt với tình huống khẩn cấp. Thay vì phản ứng ngay, bạn hãy dành chút thời gian lấy lại bình tĩnh.
Truth:  Xác định xem bạn có thể làm gì để hỗ trợ không. Dành thời gian suy nghĩ trước khi hành động. Chuẩn bị một bộ sơ cứu. Hỏi người bị thương những câu cơ bản. Tránh di chuyển người bị thương. Chỉ dùng điện thoại để tìm sự trợ giúp.
Content:  Điều quan trọng nhất bạn có thể làm là giữ bình tĩnh và tự chủ trong tình huống khẩn cấp. Đôi khi bạn không thể làm gì, và như vậy cũng không sao. Đừng bận lòng khi phải thừa nhận rằng bạn không giúp gì được.  Nếu những người khác có mặt ở hiện trường rối trí hoặc sợ hãi, bạn hãy trấn an họ và huy động mọi người cùng hỗ trợ. Tốt hơn là bạn nên ở bên cạnh người bị nạn với cử chỉ ân cần thay vì có những hành động có thể gây tổn hại thêm. Nếu không chắc phải làm gì, bạn chỉ cần ở lại với nạn nhân. Nếu có thể, bạn hãy bắt mạch cho nạn nhân, ghi nhớ các sự kiện xảy ra và hỏi v

In [ ]:
input_lines[i]


'Điều quan trọng nhất bạn có thể làm là giữ bình tĩnh và tự chủ trong tình huống khẩn cấp. Đôi khi bạn không thể làm gì, và như vậy cũng không sao. Đừng bận lòng khi phải thừa nhận rằng bạn không giúp gì được.  Nếu những người khác có mặt ở hiện trường rối trí hoặc sợ hãi, bạn hãy trấn an họ và huy động mọi người cùng hỗ trợ. Tốt hơn là bạn nên ở bên cạnh người bị nạn với cử chỉ ân cần thay vì có những hành động có thể gây tổn hại thêm. Nếu không chắc phải làm gì, bạn chỉ cần ở lại với nạn nhân. Nếu có thể, bạn hãy bắt mạch cho nạn nhân, ghi nhớ các sự kiện xảy ra và hỏi về bệnh sử của họ. Đây là những thông tin mà có thể bạn cần biết để báo lại với đội cứu hộ. Người ta thường suy nghĩ và hành động hoảng loạn khi đối mặt với tình huống khẩn cấp. Thay vì phản ứng ngay, bạn hãy dành chút thời gian lấy lại bình tĩnh. Hít thở thật sâu trước khi bắt tay vào hành động.  Sự việc thường thay đổi bất ngờ trong tình huống khẩn cấp. Đừng hoảng hốt nếu sự việc không diễn biến như bạn dự tính. Nghỉ

In [ ]:
sentence = """
Chương trình hành động của Chính phủ đã đề ra 33 nhiệm vụ tập trung vào 5 nhóm nhiệm vụ giải pháp, trước hết là nhóm nhiệm vụ nâng cao, thống nhất nhận thức về đặc thù của đô thị, vai trò, vị thế của đô thị trong sự phát triển chung. Trong đó, xác định phát triển đô thị gồm 3 trụ cột chính là quy hoạch, xây dựng và quản lý đô thị.

"Quy hoạch phải đi trước một bước với tư duy đột phá, tầm nhìn chiến lược. Quy hoạch lúng túng, chậm chạp, không được đầu tư ngang tầm thì lãng phí nguồn lực, phát triển không bền vững, phát triển không đột phá. Công tác quy hoạch làm tốt thì mới tiết kiệm được nguồn lực, xác định và phát huy được tiềm năng khác biệt, cơ hội nổi trội, lợi thế cạnh tranh, chỉ ra và hóa giải những khó khăn, thách thức của địa phương, của vùng”, Thủ tướng phân tích.

Theo Thủ tướng, quy hoạch tổng thể nhưng thực hiện có thể phân kỳ phù hợp với nguồn lực; có thể kiên trì thực hiện trong 5 năm, 10 năm, 20 năm, thậm chí hàng trăm năm… và nếu tôn trọng, làm theo quy hoạch hoàn chỉnh thì sẽ có một đô thị trật tự và phát triển.

Ngoài ra cần đầu tư phát triển hệ thống hạ tầng đô thị đồng bộ, hiện đại trên cơ sở huy động mạnh mẽ các nguồn lực, có cơ chế, chính sách để huy động nguồn lực, kết hợp hài hòa, hợp lý, hiệu quả giữa nguồn lực bên trong và bên ngoài, giữa nguồn lực Nhà nước với các nguồn lực xã hội; đồng thời tiết kiệm, tăng thu, giảm chi, phân bổ và sử dụng nguồn lực hiệu quả, đầu tư có trọng tâm, trọng điểm.

“Hiện vẫn còn nhiều vướng mắc về thể chế, cơ chế, chính sách cần xử lý nhưng với cơ chế đang có thì các địa phương vẫn có thể làm được nếu lãnh đạo các bộ, ngành, địa phương tập trung suy nghĩ, vận dụng tối đa các quy định hiện có trên tinh thần dám nghĩ, dám làm", Thủ tướng lưu ý.
Bùi Thạc Chuyên cho biết gặp áp lực khi chuyển thể văn học từ các tác phẩm của Nguyễn Ngọc Tư. Theo anh, truyện của nữ nhà văn thường có lối viết ngắn nhưng giàu tính gợi mở, các nhân vật có số phận, chứa đựng nội tâm và tiếng nói mạnh mẽ. Do đó, ngoài yếu tố sáng tạo, anh cố gắng để giữ lại những gì mình tâm đắc trong truyện gốc. 

Tro tàn rực rỡ có sự tham gia của Phương Anh Đào, Lê Công Hoàng, Bảo Ngọc Doling, Ngô Quang Tuấn, Hạnh Thúy… Phim là câu chuyện của ba phụ nữ với 3 số phận khác nhau, cách họ sống với niềm vui, nỗi ẩn ức và ngọn lửa tình yêu với chính những người đàn ông của đời mình. Bối cảnh của chuyện xảy ra ở xóm Thơm Rơm, Cà Mau với nhiều chất liệu điện ảnh và văn hóa.

Chia sẻ với VietNamNet, Phương Anh Đào cho biết đây là vai diễn thử thách lớn nhất trong sự nghiệp diễn xuất. Trong phim, diễn viên đảm nhận vai Nhàn - một trong 3 nữ chính. Nhàn có số phận tréo ngoe, luôn khao khát yêu và được yêu đến cháy bỏng. Vốn được biết đến với nét đẹp hiện đại, song nữ diễn viên đã “lột xác” ấn tượng với vai diễn người phụ nữ khắc khổ, tảo tần trong phim.

“Tôi thay đổi hình ảnh rất nhiều cho vai diễn với mong muốn mang đến một Phương Anh Đào khác trên màn ảnh. Thay vì những cảnh bầm dập thể xác, nhân vật của tôi chứa nhiều thương tổn tinh thần. Mọi cảm xúc đều được giữ ở bên trong, và nó khiến nhiều người suy ngẫm. Đó cũng là cách kể chuyện của nhân vật mà anh Bùi Thạc Chuyên muốn hướng đến trong phim”, cô chia sẻ. 

Phim ghi nhận điểm sáng với diễn xuất của các diễn viên. Họ lột tả được nhân vật của mình khi bị đẩy vào hoàn cảnh cùng cực và cách thích nghi một cách bị động trước cuộc đời gần như không lối thoát. 

Hai diễn viên nam Lê Công Hoàng và Quang Tuấn thể hiện tròn vai giữa bối cảnh câu chuyện hầu hết đều xoay quanh những người đàn bà. Diễn viên Quang Tuấn tiết chế được biểu cảm cường điệu so với các vai diễn trước đây. Anh khắc họa một gã đàn ông đau đớn mất con gái và nỗi ám ảnh thường trực vì ánh sáng rực rỡ của những ngọn lửa được đốt lên từ chính ngôi nhà mình. Nam diễn viên diễn chừng mực, không lên gân hay gào thét để người xem tự thấu cảm về nỗi đau cùng cực.

"Do quá nhập vai, tôi đã tự lấy bật lửa đốt tay mình và đến giờ vẫn còn để lại vết bỏng", anh kể. Trước ngày phim bấm máy, nam diễn viên dành một tháng để đến set quay, học nghề ở các lò than gần một tháng để đảm bảo hóa thân tốt. 

"""

text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=False
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

Thủ tướng Phạm Minh Chính nhấn mạnh việc coi trọng và đầu tư thích đáng cho công tác quy hoạch; đa dạng hóa, kết hợp, phát huy mạnh mẽ các nguồn lực, có cơ chế, chính sách để huy động mạnh mẽ các nguồn lực, kết hợp hài hòa, hợp lý, hiệu quả giữa nguồn lực bên trong và bên ngoài, giữa nguồn lực với các nguồn lực xã hội.


''